In [1]:
import GSV_utils as gsv
import os
import tempfile

In [ ]:
api_key = "your_api_key"

In [ ]:
def main(panoid, outdir, fisheyesize=1024):
    """
    Run the full process to generate a fisheye image from Google Street View tiles.

    Args:
        panoid (str): The panorama ID.
        outdir (str): Output directory for images.
        fisheyesize (int, optional): Output fisheye image size. Default is 1024.
    """
    os.makedirs(outdir, exist_ok=True)
    mosaic_path = gsv.assemble_mosaic(panoid, outdir)
    if mosaic_path:
        fisheye_path = os.path.join(outdir, "fisheye.png")
        gsv.equirectangular_to_fisheye(mosaic_path, fisheye_path, fisheyesize) #or equirectangular_to_equisolid(final_mosaic_path, fisheye_path, fisheyesize)
        print(f"Fisheye image saved at {fisheye_path}")
    else:
        print("Failed to create mosaic.")

main(gsv.get_panoid(10.050582795384798, 125.16523037305768, api_key), "./temp", fisheyesize=1024)  # Example usage

In [ ]:
def scrap_fisheye_image(api_key, region="global", fisheye_dir="fisheye", mosaic_dir="mosaic"):
    """
    Download and generate a fisheye image from Google Street View for a random land coordinate.

    This function selects a random coordinate (globally or within France), retrieves the corresponding
    Google Street View panorama, assembles a mosaic from image tiles, converts it to a fisheye projection,
    and saves the result. If no panorama is found or mosaic creation fails, it retries automatically.

    Args:
        api_key (str): Google Street View API key.
        region (str, optional): Region to sample coordinates from. "global" for worldwide, "france" for France only. Default is "global".
        fisheye_dir (str, optional): Directory to save fisheye images. Default is "fisheye".
        mosaic_dir (str, optional): Directory to save mosaic images. Default is "mosaic".

    Returns:
        str: Path to the saved fisheye image.
    """
    os.makedirs(fisheye_dir, exist_ok=True)
    os.makedirs(mosaic_dir, exist_ok=True)
    
    # Get random coordinates
    if region == "france":
        lat, lon = gsv.generate_random_around_france_coordinate()
    else:
        lat, lon = gsv.generate_random_land_coordinate()
    
    # Get panoid
    panoid = gsv.get_panoid(lat, lon, api_key)
    if not panoid:
        print("No panorama found, retrying...")
        return scrap_fisheye_image(api_key, region, fisheye_dir, mosaic_dir)
    
    # Use a temporary directory for tiles
    with tempfile.TemporaryDirectory() as temp_dir:
        # Generate mosaic
        mosaic_path = gsv.assemble_mosaic(panoid, temp_dir)
        if not mosaic_path:
            print("Failed to create mosaic, retrying...")
            return scrap_fisheye_image(api_key, region, fisheye_dir, mosaic_dir)
        
        # Move mosaic to permanent directory
        final_mosaic_path = os.path.join(mosaic_dir, f"{lat}_{lon}.png")
        os.rename(mosaic_path, final_mosaic_path)
        
        # Generate fisheye image
        fisheye_path = os.path.join(fisheye_dir, f"{lat}_{lon}.png")
        gsv.equirectangular_to_fisheye(final_mosaic_path, fisheye_path)  #or equirectangular_to_equisolid(final_mosaic_path, fisheye_path)

    print(f"Fisheye image saved at {fisheye_path}")
    return fisheye_path

for i in range(100):
    scrap_fisheye_image(api_key, region="global")